---
# **LA LOI DU CROUS :**
## *Étude des déterminations du prix des résidences universitaires Crous et de leurs répartitions dans les départements*
---

# Introduction:

350 000 étudiants habitent dans une résidence des centres régionaux des œuvres universitaires et scolaires (Crous), soit seulement 10% des 2 million d'étudiants vivant hors de leur domiciles, selon un  rapport sur le logement et la précarité des étudiants, des apprentis et des jeunes actifs de l'Assemblée nationale datant de janvier 2022 [1]. Selon ce même rapport, le logement est le premier poste de dépense des étudiants soit 60% de leurs revenus. Il est donc primordiale de donner accès à un logement abordable aux étudiants qui sont de plus en plus précaires de manières équitables selon les départements où ils se trouvent. 

Partant de ces constats, nous nous sommes posés plusieurs questions : le loyer est-il le même pour un logement situé dans un marché immobilier en tension comme Paris que dans un marché qu'il l'est moins comme Dijon ? Ou au contraire, est-ce que les prix sont globalement uniformes ? De même, est-ce que la concentration importante d'étudiants scolarisés à proximité de la résidence influe le prix vers la hausse? 

De ce fait, nous chercherons donc à répondre à la problématique suivante : 

***Les loyers des résidences des Crous suivent-ils la loi de l'offre et de la demande ?***

**Modèle :**

Ainsi, nous proposons d'étudier dans ce rapport, d'une part la répartition des 795 résidences des Crous sur le territoire; et d'autre part, les déterminants du prix du loyer de celles-ci pour pouvoir montrer d'éventuelles variations entre les départements. Pour expliquer le loyer nous utiliserons un modèle d'offre et de demande économique élèmentaire. Dans celui-ci, la résidence offre des biens de surfaces variables q à un prix p en mètre carré, et fait face à une concurrence représenté par le montant des loyers dans le parc privé. La demande sera représentée par l'effectifs d'étudiants scolarisés dans une école se situant dans un rayon de 20km de la résidence. Nous faisons l'hypothèse que les résidences proposent à peu près les mêmes caractéristiques entre elles et que les étudiants sont des price-takers. 

Nous réaliserons ainsi des régressions linéaires classiques : 

\begin{equation}
\hat{Y}_i = \hat{\beta}_0 + \hat{\beta}_1 X_i + \hat{\epsilon}_i
\end{equation}

**PLAN:**

1. Dans une première partie, nous réccupérons une base de données avec les loyers (maximum, minimum et moyen), les surfaces (idem), les coordonnées géographiques, les loyers privés dans la ville de la résidence et l'effectif des étudiants dans un périmètre de 20 kilomètres. 

2. Dans une deuxième partie, nous analyserons d'une part la répartition des résidences en France et, d'autre part, nous réaliserons des statistiques descriptives sur les différentes variables de notre base de données. 

3. Dernièrement, nous utiliserons des modèles de régressions linéaires pour répondre à notre problématique. 


## Les données utilisées 

Nous nous servirons de plusieures base de données : 

1. **Ensemble des logements proposés aux étudiants par le réseau des CROUS :** 

Données mises à disposition par le Centre National des Œuvres Universitaires et Scolaires (CNOUS). Ils ont été publiés en 2017 et sont mises à jour assez régulièrement. Le problème principale de cette base de données est la récupération des loyers et des surfaces qui sont stockés dans une colonne nommée "infos" (s'il y en a). De plus, chaque colonne a été remplies par les résidences. La qualité et la structure des informations sont donc très variables. Nous avons donc mis en place des techniques de traitements textuels ou de natural language processing (NPL).

https://www.data.gouv.fr/fr/datasets/ensemble-des-logements-proposes-aux-etudiants-par-le-reseau-des-crous/

2. **"Carte des loyers" - Indicateurs de loyers d'annonce par commune en 2018**

Données obtenues par une collaboration entre la Direction Générale de l’Aménagement, du Logement et de la Nature (DGALN), une équipe de recherche en économie d’Agrosup Dijon, de l’Institut national de la recherche en agronomique (INRAE), de SeLoger, de leboncoin et de PAP. Ces équipes ont créé différent indicateurs par communes à partir de 9 million d'annonces. Les données datent de 2018 et nous nous servirons uniquement de données pour les appartements.

https://www.data.gouv.fr/fr/datasets/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2018/


3. **Statistiques sur les effectifs d'étudiants inscrits par établissement public sous tutelle du ministère en charge de l'Enseignement supérieur (avec doubles inscriptions CPGE)**

"Ce jeu de données propose un ensemble de statistiques sur les effectifs d'étudiants inscrits de 2006-07 à 2021-22 par établissement public sous tutelle du ministère en charge de l'Enseignement supérieur français : universités, universités de technologie, grands établissements, COMUE, écoles normales supérieures, écoles centrales, INSA, autres écoles d'ingénieurs..."

https://www.data.gouv.fr/fr/datasets/statistiques-sur-les-effectifs-detudiants-inscrits-par-etablissement-public-sous-tutelle-du-ministere-en-charge-de-lenseignement-superieur-avec-doubles-inscriptions-cpge/

4. **Contours Communes France Administrative "Format Admin-Express" avec arrondissements**

Jeu de données de l'IGN regroupant les contours communales avec arrondissements.

https://www.data.gouv.fr/fr/datasets/contours-communes-france-administrative-format-admin-express-avec-arrondissements/

5. **Contours des départements français issus d'OpenStreetMap**

Jeu de données d'OpenStreetMap regroupant les contours géographiques des départements. 

https://www.data.gouv.fr/fr/datasets/contours-des-departements-francais-issus-d-openstreetmap/





# Librairies nécessaires

In [1]:
%%capture 
#hide output 
!pip install geoplot
!pip install contextily
!pip install altair

In [2]:
import pandas as pd 
import geopandas as gpd 
import nltk
nltk.download('punkt')
import contextily as ctx
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import altair as alt 

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


# Partie I : Obtention du dataframe 

L'objectif de cette partie est de créer une base de données contenant pour chaque résidence : 
- Le prix du loyer (maximum, minimum et moyen);
- La surface (idem);
- Les coordonnées géographiques compatibles avec Géopanda (nécessaires pour réccupèrer le nombre d'étudiants et les cartes);
- L'indicateur du loyer privé maximum d'un appartement dans la ville de la résidence;
- Le nombre d'étudiants scolarisés dans un rayon de 20 km autour de la résidence hors CPGE

Le principale problème de cette base de données est le fait que d'une part, il n'existe pas de colonne contenant exlicitement le loyer et les surfaces, et d'autre part, chaque résidence a remplie à sa manière les informations sur les loyers et les surfaces (s'ils ont rempli). Nous avons donc mis en place des fonctions de Natural Language Processing (NLP) pour pouvoir les réccupèrer (pour plus d'informations voire le document : functions_for_data_cleaning.py) 

## I.1. Récupération des loyers, des surfaces et de la localisation des résidences

In [3]:
"""
Importation des données.
"""
df = pd.read_csv("https://data.enseignementsup-recherche.gouv.fr//explore/dataset/fr_crous_logement_france_entiere/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false", 
                 sep =';')

In [4]:
"""
Réduction de notre base de données aux 5 colonnes qui nous intéressent (nom de la résidence ou "title";
la colonne comportant toutes les infos sur les surfaces et les loyers ; l'adresse; les coordonnées géographiques; 
et les noms des régions.
"""
df = df[["title", "infos", "address" ,"geocalisation", "regions"]]

In [5]:
from functions_for_data_cleaning import * #document .py comportant nos fonctions nécessaires au nettoyage

df = get_loyer(df) #récuppération des loyers à partir de la colonne "infos"
df = get_surface(df) #récuppération des surfaces à partir de la colonne "infos"
df = get_localisation(df) #récuppération des coordonnées géographiques à partir de la colonne "geocalisation"

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## I.2. Récupération des loyers privés des communes (avec la géolocalisation des communes)

Le but de cette partie est de joindre la base précédemment obtenue avec la base des indicateurs des loyers privés maximum pour un appartement et par communes (var : upr.lPm2)

In [6]:
df_private = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da", encoding= 'unicode_escape', sep = ";")

On ne garde que le code INSEE, le nom de la ville et le loyer supérieur par m^2


In [7]:
df_private = df_private[["INSEE", "LIBGEO","upr.IPm2"]]

On renomme la colonne INSEE en codgeo pour pouvoir la merger avec la carte des contours des villes.

In [8]:
df_private.rename(columns = {'INSEE':'codgeo',"upr.IPm2" : "Loyer_prive_m2" }, inplace = True)

In [9]:
"""
On rajoute un 0 devant le code INSEE quand il ne fait que quatre chiffres (pour uniformiser avec l'autre base).
"""
df_private["codgeo"] = df_private["codgeo"].apply(lambda x : "0" + x if len(x) == 4 else x)


df_private["Loyer_prive_m2"] = df_private["Loyer_prive_m2"].apply(lambda x : x.replace(',','.'))

In [10]:
df_private = df_private.set_index('codgeo') #on réindex le dataframe avec le code INSEE pour merger après. 

On récuppère les contours géographique des communes pour pouvoir faire la jointure avec notre base.(prends du temps)

In [11]:
gdf_communes_boundaries = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/e9391593-fa95-4153-aabe-87ca84d197e9")

On ne garde que le code INSEE et les données polygoniales. 

In [12]:
gdf_communes_boundaries.rename(columns = {'INSEE_COM':'codgeo'}, inplace = True)
gdf_communes_boundaries = gdf_communes_boundaries[["codgeo", "geometry"]]
gdf_communes_boundaries = gdf_communes_boundaries.set_index('codgeo')


On concatène les deux datafarmes

In [13]:
gdf_private = pd.concat([df_private,gdf_communes_boundaries], axis = 1, join = "inner")

## I.3. Jointures spatiales

Maintenant que nous avons une base des loyers privés avec les coordonnées des communes nous allons pouvoir réaliser une jointure spatiale avec la base des résidences.

In [14]:

#gdf_private['geometry'] = gdf_private['geometry'].apply(wkt.loads) #on transforme en format wkt (sinon ça ne marche pas)
gdf_private = gpd.GeoDataFrame(gdf_private)  #on transforme en geodataframe 
df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within 
#i.e si le point est dans le polynôme. 

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_270/2580680376.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within


In [15]:
del df['index_right'] #on supprime l'index right pour la prochaine jointure

## I.4 Ajout des départements 

Pour pouvoir avoir des comparaisons par départements nous allons réccupèrer les départements et les joindre avec nos bases grâce à une jointure spatiale. 

In [16]:
gdf_dep = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/eb36371a-761d-44a8-93ec-3d728bec17ce")
gdf_dep.rename(columns = {'nom':'departement'}, inplace = True)
gdf_dep = gdf_dep[["departement", "geometry"]]

In [17]:
df = gpd.sjoin(df, gdf_dep, how='left', op='within')
del df['index_right'] #on supprime l'index right pour la prochaine jointure

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_270/3363061827.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, gdf_dep, how='left', op='within')


## I.5. Association de résidences à une école 

Import de notre base de données des écoles pour pouvoir réccupèrer le nombre d'étudiants scolarisés dans un rayon de 20km.

In [18]:
df_schools = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/0c713161-26fb-415e-ac1d-8769125f338d", sep = ";")

In [19]:
df_schools = df_schools[df_schools["annee_universitaire"] == "2021-22" ] #on ne prend que l'année 2021-22
df_schools = df_schools[["etablissement_lib", "effectif_sans_cpge", "etablissement_code_commune", "etablissement_commune"]]
df_schools.rename(columns = {'etablissement_code_commune':'codgeo'}, inplace = True) #On renomme la colonne pour la jointure

On enlève les valeurs manquantes (il manque des coordonnées pour certains établissments).

In [20]:
index_with_nan = df_schools.index[df_schools.isnull().any(axis=1)]
df_schools.drop(index_with_nan,0, inplace=True) 
df_schools.set_index('codgeo', inplace = True) #on recommence l'index

/tmp/ipykernel_270/2476089590.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_schools.drop(index_with_nan,0, inplace=True)


Jointure avec les coordonnées géographiques des communes pour pouvoir trouver dans un rayon de 20km les écoles autour des résidences.

In [21]:
df_schools = df_schools.join(gdf_communes_boundaries)
df_schools = gpd.GeoDataFrame(df_schools)
df_schools.crs = "epsg:4326"

On applique la fonction get_nb_student disponible dans le fichier function_for_data_cleaning.py qui permet d'obtenir le nombre d'étudiants dans un rayon de 20km autour de nos résidences.

In [22]:
df.crs = "epsg:4326" #on définit la projection de notre dataframe
df = get_nb_student(df, df_schools, 20)

## I.6. Obtention de la base de données finale

In [23]:
df 

,title,infos,address,geocalisation,regions,Loyer,Max Loyer,Min Loyer,Mean Loyer,Surface,...,Min Surface,Mean Surface,Longitude,Latitude,geometry,LIBGEO,Loyer_prive_m2,departement,Schools,Nbstudents
0,Cité U' La Bourgeonnière,Localisation\n\r\n Tramway Ligne 2 (arrêt Bour...,"5, Rue des Renards, 44322 Nantes","47.2536315918,-1.5635741949",Pays de la Loire,"[164.95, 254.8, 346.8, 378.8]",378.80,164.95,286.337500,"[9.0, 9.0, 13.0, 18.0, 9.0, 9.0, 13.0, 18.0]",...,9.0,12.250000,-1.563574,47.253632,POINT (-1.56357 47.25363),Nantes,15.32628634,Loire-Atlantique,[Nantes Université],44394
1,Résidence U' Les Saumonières,Localisation\n\r\n Tramway Ligne 2 (arrêt Rect...,"14, Rue des Saumonières, 44300 Nantes","47.2551040649,-1.5516899824",Pays de la Loire,[361.46],361.46,361.46,361.460000,"[17.0, 19.0]",...,17.0,18.000000,-1.551690,47.255104,POINT (-1.55169 47.25510),Nantes,15.32628634,Loire-Atlantique,[Nantes Université],44394
2,Résidence U' Corbilo,Localisation\n\r\n Tram 2-3 (arrêt Mangin)Bus ...,"18, rue Anatole de Monzie, 44200 Nantes","47.2025222778,-1.5433219671",Pays de la Loire,"[343.79, 407.43, 558.75]",558.75,343.79,436.656667,"[20.0, 30.0, 30.0, 35.0, 20.0, 30.0]",...,20.0,27.500000,-1.543322,47.202522,POINT (-1.54332 47.20252),Nantes,15.32628634,Loire-Atlantique,[Nantes Université],44394
3,Résidence U' Ile de Nantes,Localisation\n\r\n Bus C5 et 26 (arrêt Républi...,"23, Bd de la Prairie au Duc, 4200 Nantes","47.2048683167,-1.5581860542",Pays de la Loire,"[338.21, 358.71]",358.71,338.21,348.460000,"[18.0, 27.0, 18.0, 27.0]",...,18.0,22.500000,-1.558186,47.204868,POINT (-1.55819 47.20487),Nantes,15.32628634,Loire-Atlantique,[Nantes Université],44394
4,Résidence U' Longchamp,Localisation\n\r\n Tramway 3 (arrêt Longchamp)...,"79, Rue de la Gaudinière, 44300 Nantes","47.2384872437,-1.5774370432",Pays de la Loire,"[309.15, 326.23, 360.49]",360.49,309.15,331.956667,"[18.0, 28.0, 18.0, 22.0, 28.0]",...,18.0,22.800000,-1.577437,47.238487,POINT (-1.57744 47.23849),Nantes,15.32628634,Loire-Atlantique,[Nantes Université],44394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,Cité de Lébisey (secteur Caen),114-116 rue de lébisey 14070 Caen Cedex 5\r\n ...,114-116 rue de lébisey 14070 Caen Cedex 5,"49.1998634338,-0.3526327014",Normandie,"[235.1, 254.85, 357.3, 429.3]",429.30,235.10,319.137500,"[9.0, 9.0, 17.0, 20.0]",...,9.0,13.750000,-0.352633,49.199863,POINT (-0.35263 49.19986),Caen,14.34459394,Calvados,[École nationale supérieure d'ingénieurs de Ca...,32933
791,Résidence Erik Satie - Lébisey (secteur Caen),114-116 rue de Lébisey - 14070 Caen cedex 5\r\...,114-116 rue de Lébisey - 14070 Caen cedex 5,"49.20095743696,-0.35141229629517",Normandie,"[348.5, 406.2]",406.20,348.50,377.350000,"[20.0, 30.0]",...,20.0,25.000000,-0.351412,49.200957,POINT (-0.35141 49.20096),Caen,14.34459394,Calvados,[École nationale supérieure d'ingénieurs de Ca...,32933
792,Residence Bacot (secteur Caen),10 bd Maréchal Juin - BP 8515314070 Caen cedex...,10 bd Maréchal Juin - BP 85153 14070 Caen cedex 5,"49.2125244141,-0.3707295656",Normandie,"[396.54, 470.56]",470.56,396.54,433.550000,"[20.0, 30.0, 50.0]",...,20.0,33.333333,-0.370730,49.212524,POINT (-0.37073 49.21252),Caen,14.34459394,Calvados,[École nationale supérieure d'ingénieurs de Ca...,32933
793,Résidence Lecesne (secteur du Havre),"Description\n\nEquipement : kitchenette, évier...",124 rue Jules Lecesne 76600 LE HAVRE,"49.4935112,0.11666479999997",Normandie,[363.0],363.00,363.00,363.000000,[18.0],...,18.0,18.000000,0.116665,49.493511,POINT (0.11666 49.49351),Le Havre,13.93122446,Seine-Maritime,[Université Le Havre Normandie],7355
